## Analytics: Exploratory Data Analysis

This notebook contains the analysis of the IBM HR dataset sourced from: pavansubhash on Kaggle.com to answer key business questions. We will use both Python *(with Pandas)* and SQL to perform the analysis.

 **Assumptions**: The hr_database.db file has been created using the setup_database.py script.

**Setup**: Connecting to the Database
First, let's establish a connection to our SQLite database.

In [ ]:
import pandas as pd
import sqlite3

# Path to your SQLite database
db_path = 'hr_database.db'

# Create a connection
conn = sqlite3.connect(db_path)

# Function to run SQL queries and return a DataFrame
def sql_query(query):
    return pd.read_sql_query(query, conn)
    


### Q1: How many total employees are there?


**Pandas Approach**
We can load the entire table into a DataFrame and find its length.


In [ ]:
df = pd.read_sql_table('employees', conn)
total_employees = len(df)
print(f"Total number of employees: {total_employees}")


**Result**: Total number of employees: 1470


**SQL Approach**
A simple COUNT(*) query will give us the answer.


In [ ]:
SELECT COUNT(*) as total_employees FROM employees;


In [ ]:

result = sql_query("SELECT COUNT(*) as total_employees FROM employees;")
print(result)


**Result**:
| total_employees |
|-----------------|
| 1470            |




### Q2: What is the employee count for each department?


**Pandas Approach**

In [ ]:
department_counts = df['Department'].value_counts().reset_index()
department_counts.columns = ['Department', 'EmployeeCount']
print(department_counts)

**SQL Approach**

In [ ]:
SELECT
    Department,
    COUNT(*) as EmployeeCount
FROM
    employees
GROUP BY
    Department
ORDER BY
    EmployeeCount DESC;

In [ ]:
result = sql_query("""
    SELECT Department, COUNT(*) as EmployeeCount
    FROM employees
    GROUP BY Department
    ORDER BY EmployeeCount DESC;
""")
print(result)

**Insight**: The Research & Development department is by far the largest, followed by Sales, and then Human Resources.



### Q3: What is the average monthly income by job role?


**Pandas Approach**

In [ ]:
avg_income_by_role = df.groupby('JobRole')['MonthlyIncome'].mean().round(2).sort_values(ascending=False).reset_index()
print(avg_income_by_role)


**SQL Approach**

In [ ]:
SELECT
    JobRole,
    AVG(MonthlyIncome) as AverageMonthlyIncome
FROM
    employees
GROUP BY
    JobRole
ORDER BY
    AverageMonthlyIncome DESC;


In [ ]:

result = sql_query("""
    SELECT JobRole, ROUND(AVG(MonthlyIncome), 2) as AverageMonthlyIncome
    FROM employees
    GROUP BY JobRole
    ORDER BY AverageMonthlyIncome DESC;
""")
print(result)




**Insight**: Manager and Research Director roles have the highest average monthly incomes, while Research Scientist and Laboratory Technician are on the lower end

### Q4: Who are the top 5 employees by performance rating?

**Note**: We need a unique identifier. We'll use the EmployeeID that was added during database setup.



**Pandas Approach**

In [ ]:

top_5_performers = df.sort_values(by='PerformanceRating', ascending=False).head(5)
print(top_5_performers[['EmployeeID', 'JobRole', 'PerformanceRating']])


**SQL Approach**

In [ ]:
SELECT
    EmployeeID,
    JobRole,
    PerformanceRating
FROM
    employees
ORDER BY
    PerformanceRating DESC
LIMIT 5;


In [ ]:

result = sql_query("""
    SELECT EmployeeID, JobRole, PerformanceRating
    FROM employees
    ORDER BY PerformanceRating DESC
    LIMIT 5;
""")
print(result)



**Insight**: This allows us to quickly identify high-performing individuals who might be candidates for recognition or promotion.



### Q5: Which department has the highest average performance ratin?

**Pandas Approach**

In [ ]:

avg_perf_by_dept = df.groupby('Department')['PerformanceRating'].mean().sort_values(ascending=False).reset_index()
print(avg_perf_by_dept)


**SQL Approach**

In [ ]:
SELECT
    Department,
    AVG(PerformanceRating) as AveragePerformanceRating
FROM
    employees
GROUP BY
    Department
ORDER BY
    AveragePerformanceRating DESC;


In [ ]:

result = sql_query("""
    SELECT Department, AVG(PerformanceRating) as AveragePerformanceRating
    FROM employees
    GROUP BY Department
    ORDER BY AveragePerformanceRating DESC;
""")
print(result)

**Insight**: While the differences are subtle, the Sales department has a slightly higher average performance rating.

### Additional Business Questions


#### Q6: How does overtime affect attrition rates across different departments?


**SQL Approach**: We can use a CASE statement to count attrition and calculate the rate

In [ ]:
SELECT
    CASE
        WHEN YearsAtCompany <= 2 THEN '0-2 Years'
        WHEN YearsAtCompany <= 5 THEN '3-5 Years'
        WHEN YearsAtCompany <= 10 THEN '6-10 Years'
        ELSE '10+ Years'
    END as TenureBucket,
    JobSatisfaction,
    ROUND( (SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) as AttritionRate
FROM
    employees
GROUP BY
    TenureBucket, JobSatisfaction
ORDER BY
    TenureBucket, JobSatisfaction;


In [ ]:

result = sql_query("""
    SELECT
        CASE
            WHEN YearsAtCompany <= 2 THEN '0-2 Years'
            WHEN YearsAtCompany <= 5 THEN '3-5 Years'
            WHEN YearsAtCompany <= 10 THEN '6-10 Years'
            ELSE '10+ Years'
        END as TenureBucket,
        JobSatisfaction,
        ROUND( (SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) as AttritionRate
    FROM
        employees
    GROUP BY
        TenureBucket, JobSatisfaction
    ORDER BY
        TenureBucket, JobSatisfaction;
""")
print(result)


**Insight**: Attrition is highest among employees with low job satisfaction (level 1), especially within their first two years at the company (a staggering 44.5% attrition rate). As tenure increases, low job satisfaction becomes a less pronounced driver of attrition, though it remains a factor.



#### Q8: Is there a correlation between education level and promotion to a higher job level?


**SQL Approach**: We can analyze the average job level for each field of education.

In [ ]:
SELECT
    EducationField,
    ROUND(AVG(JobLevel), 2) as AverageJobLevel,
    ROUND(AVG(MonthlyIncome), 2) as AverageMonthlyIncome
FROM
    employees
GROUP BY
    EducationField
ORDER BY
    AverageJobLevel DESC;


In [ ]:

result = sql_query("""
    SELECT
        EducationField,
        ROUND(AVG(JobLevel), 2) as AverageJobLevel,
        ROUND(AVG(MonthlyIncome), 2) as AverageMonthlyIncome
    FROM
        employees
    GROUP BY
        EducationField
    ORDER BY
        AverageJobLevel DESC;
""")
print(result)


**Insight**: Employees with a background in *'Marketing'* and *'Life Sciences'* tend to reach slightly higher average job levels and incomes compared to other fields. *'Human Resources'* and *'Technical Degree'* fields have the lowest average job levels. This could inform recruitment and development strategies.